# Purpose of this notebook:
The purpose of this notebook is to familiarize the user with the functionality of the ptsa object TalReader, used to extract out relevant electrode localization arrays, and relevant changes that have been made to it recently.

Let's start by importing the TalReader and JsonIndexReader, which allows us to get the relevant path for the TalReader (and EEGReader, BaseEventReader, etc.):

In [1]:
# General imports
import numpy as np
import os, sys

# Relevant import line
from ptsa.data.readers import JsonIndexReader, TalReader

ModuleNotFoundError: No module named 'ptsa'

# There are two ways to access data on 

1. Mount rhino locally 
2. Through accessing the cluster directly

This is relevant because typically rhino gets mounted at '/Volumes/rhino', which changes precisely where the relevant paths are. Below we'll use the package os in order to check which directory we're in and then adjust are paths using that check.

In [2]:
cwd = os.getcwd()
local = '' if (cwd.split('/')[1][:4] == 'home') else '/Volumes/rhino'
protocol = local + '/protocols/r1.json'

# Initalize the json reader by passing it the protocol path
jr = JsonIndexReader(protocol)

# Use JsonIndexReader to get subjects

In [3]:
# By default jr.aggregate_values will return a set
subjects_set = jr.aggregate_values('subjects',experiment='FR1')
print('The default output type of JsonIndexReader is', type(subjects_set))


# We can convert it to an array (first cast it as a list, because 
# Again the output is a set) or simply as a list like below
subjects_list = list(subjects_set)


# Remember that Arrays of sets are not the same as arrays of lists
subjects_arr = np.array(list(jr.aggregate_values('subjects',experiment='FR1')))
subjects_arr == np.array(jr.aggregate_values('subjects',experiment='FR1'))

('The default output type of JsonIndexReader is', <type 'set'>)


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

# If the user isn't sure of the experiment fields they can input, they can check all possible experiments like so:

In [4]:
# If the user isn't sure of the experimental fields they can input, they can check like so:
list(jr.aggregate_values('experiments'))

[u'catFR2',
 u'PS2.1',
 u'catFR1',
 u'catFR6',
 u'THR',
 u'catFR5',
 u'PS5_catFR',
 u'PS2',
 u'PS3',
 u'PS1',
 u'THR1',
 u'PAL1',
 u'YC2',
 u'YC1',
 u'PAL2',
 u'TH3',
 u'TH1',
 u'FR3',
 u'FR2',
 u'FR1',
 u'FR6',
 u'FR5',
 u'catFR3',
 u'PAL3',
 u'PAL5']

In [5]:
# Remember that Arrays of sets are not the same as arrays of lists
subjects_arr = np.array(list(jr.aggregate_values('subjects',experiment='FR1')))
subjects_arr == np.array(jr.aggregate_values('subjects',experiment='FR1'))

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

# Create a Function to Load subject's tal

In [6]:
# Functions to load talirach
def get_subj_tal(subject, experiment, return_channels=False):
    """Returns a subject's talairach using TalReader and JsonIndexReader
    -----
    INPUTS:
    -----
    subject: str, subject ID, e.g. 'R1111M'
    experiment: str, experiment, e.g. 'FR1', 'catFR1'
    return_channels: bool, default = False, whether to return arrays of
                     monopolar and bipolar channels used for EEGReader
    ------
    OUTPUTS if return_channels is False:
    ------
    tal_reader.read(): np.recarray, an array containing relevant values 
                       for electrode localization
    ------
    OUTPUTS if return_channels is True:
    ------                   
    mp: np.recarray, monopolar channels used for EEGReader
    bp: np.recarray, bipolar channels used for EEGReader
    tal_reader.read(): np.recarray, an array containing relevant values 
                       for electrode localization
    """
    
    # Check if we're using this locally or through rhino directly
    cwd = os.getcwd()
    local = '' if (cwd.split('/')[1][:4] == 'home') else '/Volumes/rhino'
    protocol = local + '/protocols/r1.json'
    
    # Load the protocol 
    jr = JsonIndexReader(protocol)
    
    # Get the path for the TalReader
    pairs_path = jr.get_value('pairs', subject=subject, experiment=experiment)
    
    # Create an instance of the TalReader object
    tal_reader = TalReader(filename=pairs_path)
    
    # If desired, also return the monopolar and bipolar channels
    if return_channels:
        mp = tal_reader.get_monopolar_channels()
        bp = tal_reader.get_bipolar_pairs()
        return mp, bp, tal_reader.read()
    
    # Otherwise just return the tal
    return tal_reader.read()

# Load a single subject

In [7]:
subj, exp = subjects_arr[0], 'FR1'

mp, bp, tal = get_subj_tal(subject=subj, experiment=exp, return_channels=True)

/Users/loganfickling/anaconda/envs/ptsa_testing/lib/python2.7/site-packages/numpy/core/records.py:507: FutureWarning: Numpy has detected that you may be viewing or writing to an array returned by selecting multiple fields in a structured array. 

This code may break in numpy 1.13 because this will return a view instead of a copy -- see release notes for details.
  return obj.view(dtype=(self.dtype.type, obj.dtype))


Let's start by examining the dtype of the array, as we can see TalReader.read() outputs a structured nested record array with relevant information shown below

These fields refer to the following:

'channel_1' : int, representing the first channel in the bp montage 

'channel_2' : int, representing the second channel in the bp montage 

'code' : str, the name of the bipolar channel (e.g. LF1-LF2)

'id' : ???

'is_explicit' : ???

'is_stim_only' : corresponds to the tal_stim_only struct in matlab, means that the channel was only used to stimulate and was not used for recording

'type_1' : the type of the first channel in the bp montage (D = Depth, S= Strip, G=Grid)

'type_2' : the type of the second channel in the bp montage (D = Depth, S= Strip, G=Grid)

'atlases' : numpy.record.array, an array of possible brain atlases to use for the subject

In [8]:
print('example bipolar channel:\n')
print(tal[0])

print('\ndtype names:\n')
print(tal.dtype.names)

example bipolar channel:

(1, 2, u'1LD1-1LD2', u'1ld.1-1ld.2', False, False, u'D', u'D', ((u'avg', u'None', -16.785, -13.85, -17.675), (u'avg.dural', u'None', -16.785, -13.85, -17.675), (u'das', u'CA1', nan, nan, nan), (u'ind', u'None', -18.665, 13.495, -19.995), (u'ind.dural', u'None', -18.665, 13.495, -19.995), (u'stein', u'Left CA1', nan, nan, nan), (u'tal', u'None', -17.60525, -8.807725, -15.891), (u'wb', u'Left Hippocampus', nan, nan, nan)))

dtype names:

('channel_1', 'channel_2', 'code', 'id', 'is_explicit', 'is_stim_only', 'type_1', 'type_2', 'atlases')


In order to access localization information we refer to the field 'atlases', which has the following atlases associated with it:

| Name | Description |
| --- | --- |
| `avg` | localization in average subject space (Freesurfer space?)|
| `avg.dural` |localization in average subject space whereby each electrode is snapped to the closest cortical surface (Freesurfer space?)|
| `das` | localization done by Sandy (similiar to stein but done by sandy) |
| `ind` | localization in individual subject space (Freesurfer space?) |
| `ind.dural` | localization in individual subject space whereby each electrode is snapped to the closest cortical surface (Freesurfer space?) |
| `stein` | localization done by joel stein, analgous to old TalReader's locTag field |
| `tal` | localization in talairach space 
| `wb` | localization in mni wholebrain |
| `dk` | ?? <- Not all subjects have this? |

In [9]:
# Show the atlases we can look at
tal['atlases'].dtype.names

('avg', 'avg.dural', 'das', 'ind', 'ind.dural', 'stein', 'tal', 'wb')

Let's choose to look at the individual subject atlas, but the following fields should be associated with each atlas:

| Name | Description |
| --- | --- |
| `id` | refers to the atlas id, for example the 'tal' atlas has id 'tal', 'avg' has id 'avg' etc. |
| `region` | what region in the brain the electrode is in |
| `x` | x coordinate of electrode |
| `y` | y coordinate of electrode |
| `z` | z coordinate of electrode |

In [10]:
# Fields we can access inside this atlas
tal['atlases']['ind'].dtype.names

('id', 'region', 'x', 'y', 'z')

Now we can quickly manipulate our data using this array

## Getting all coordinates

In [11]:
def get_coords(tal, atlas):
    """Returns the coordinates of the associated atlas for the tal array"""
    coords = np.vstack((tal['atlases'][atlas]['x'], 
                        tal['atlases'][atlas]['y'], 
                        tal['atlases'][atlas]['z'])).T
    return coords

In [12]:
print('ind', get_coords(tal, 'ind')[:5])

print('avg:', get_coords(tal, 'avg')[:5])

print('stein:', get_coords(tal, 'stein')[:5])

print('tal:', get_coords(tal, 'tal')[:5])

('ind', array([[-18.665,  13.495, -19.995],
       [-23.495,  12.92 , -20.92 ],
       [-28.32 ,  12.345, -21.845],
       [-33.395,  11.29 , -22.705],
       [-38.47 ,  10.235, -23.565]]))
('avg:', array([[-16.785, -13.85 , -17.675],
       [-22.255, -14.48 , -18.34 ],
       [-27.72 , -15.105, -19.   ],
       [-33.465, -16.23 , -19.525],
       [-39.215, -17.36 , -20.05 ]]))
('stein:', array([[ nan,  nan,  nan],
       [ nan,  nan,  nan],
       [ nan,  nan,  nan],
       [ nan,  nan,  nan],
       [ nan,  nan,  nan]]))
('tal:', array([[-17.60525 ,  -8.807725, -15.891   ],
       [-23.0165  ,  -9.52845 , -16.38325 ],
       [-28.4278  , -10.24916 , -16.8755  ],
       [-34.1166  , -11.46135 , -17.2087  ],
       [-39.8053  , -12.67355 , -17.54185 ]]))


## Getting all regions

In [13]:
tal['atlases']['stein']['region']

array([u'Left CA1', u'Left Amy', u'Left CA1', u'nan', u'nan', u'nan',
       u'nan', u'nan', u'Left Middle Temporal Gyrus', u'Left MTL WM',
       u'Left CA1', u'Left DG', u'nan', u'nan', u'nan', u'nan',
       u'Left Middle Temporal Gyrus', u'Left Middle Temporal Gyrus',
       u'Right Amy', u'Right Amy', u'nan', u'nan', u'nan', u'nan', u'nan',
       u'nan', u'Right DG', u'Right CA1', u'nan', u'nan', u'nan', u'nan',
       u'Right CA1', u'Right CA1', u'nan', u'nan', u'nan', u'nan', u'nan',
       u'Right MTL WM', u'Right CA1', u'Right CA1', u'nan', u'nan', u'nan',
       u'nan', u'Right Middle Temporal Gyrus',
       u'Right Middle Temporal Gyrus', u'nan', u'nan', u'nan', u'nan',
       u'nan', u'nan', u'nan', u'nan', u'nan', u'nan', u'nan', u'nan',
       u'Right Middle Temporal Gyrus', u'nan', u'nan', u'nan', u'nan',
       u'nan', u'nan', u'nan', u'nan', u'nan', u'nan', u'nan', u'nan',
       u'nan', u'nan'],
      dtype='<U256')

## Check hemisphere

In [14]:
def check_hemi(tal, atlas='ind'):
    return np.array(['Left' if x ==-1. else 'Right' for x in np.sign(tal['atlases'][atlas]['x'])])

check_hemi(tal=tal)

array(['Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left',
       'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left',
       'Left', 'Left', 'Right', 'Right', 'Right', 'Right', 'Right',
       'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right',
       'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right',
       'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right',
       'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right',
       'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right',
       'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right',
       'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right',
       'Right', 'Right', 'Right'],
      dtype='|S5')